In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [7]:
# get the data
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wiki_link).text

# parse the data
soup = BeautifulSoup(raw_wikipedia_page,'xml')

# find table
table = soup.find('table')

In [8]:

PostalCode = []
Borough = []
Neighbourhood = []

tab = []
count = 0


for row in table.findAll("tr"):
    for cells in row.findAll("td"):
        if count == 0:
            PostalCode.append(cells.text)
            count += 1
        elif count == 1:
            Borough.append(cells.text)
            count += 1
        elif count == 2:
            Neighbourhood.append(cells.text.rstrip()) 
            count = 0

PostalCodeNew = []
BoroughNew = []
NeighbourhoodNew = []

#remove "Not assigned" rows
for index in range(len(Borough)):
    if Borough[index] != "Not assigned":
        PostalCodeNew.append(PostalCode[index])
        BoroughNew.append(Borough[index])
        if Neighbourhood[index] != "Not assigned":
            NeighbourhoodNew.append(Neighbourhood[index])
        else:
            NeighbourhoodNew.append(Borough[index])

#combine postal codes
PostalCode_list = []
Borough_list = []
Neighbourhood_list = []

for index in range(len(PostalCodeNew)):
    if PostalCodeNew[index] not in PostalCode_list:
        PostalCode_list.append(PostalCodeNew[index])
        Borough_list.append(BoroughNew[index])
        Neighbourhood_list.append(NeighbourhoodNew[index])
    else:
        Neighbourhood_list[PostalCode_list.index(PostalCodeNew[index])] +=  ", " + NeighbourhoodNew[index]

            

In [9]:
df=pd.DataFrame({'PostalCode':PostalCode_list,'Borough':Borough_list,'Neighborhood':Neighbourhood_list})
df=df[['PostalCode','Borough','Neighborhood']]
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [10]:
df.shape

(103, 3)

In [11]:
#!conda install -c conda-forge geocoder --yes 
import geocoder

In [12]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(PostalCode_list[0]))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

#it does not work...

In [13]:
import pandas as pd

url="https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
c=pd.read_csv(url)

In [16]:
#print(c['Postal Code'])
#a = c.loc[c['Postal Code'] == 'M4B']
#latitude = (a.iloc[0,1])
#longitude = (a.iloc[0,2])

#print(latitude)

In [17]:
#merge coordinates info
latitude_list =[]
longitude_list = []

for index in range(len(PostalCode_list)):
    a = c.loc[c['Postal Code'] == PostalCode_list[index]]
    latitude_list.append(a.iloc[0,1])
    longitude_list.append(a.iloc[0,2])
 

dfc=pd.DataFrame({'PostalCode':PostalCode_list,'Borough':Borough_list,'Neighborhood':Neighbourhood_list,'Latitude':latitude_list,'Longitude':longitude_list})

dfc.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
